In [38]:
# Authors: Nicholas C. Firth <ncfirth87@gmail.com>
# License: TBC
from kde_ebm import mixture_model
from kde_ebm import mcmc
# from kde_ebm import plotting
from kde_ebm import datasets
from matplotlib import pyplot as plt

import warnings
# import scipy.optimize as opt
from scipy.stats import kendalltau
import matplotlib as mpl
import json

# Use a non-interactive backend
mpl.use('Agg')  # 'Agg' is a backend for non-GUI environments

# Suppress specific warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='scipy.optimize')
# warnings.filterwarnings('ignore', category=UserWarning, module='ebm.mcmc')


In [39]:
ns = [50, 200, 500]
rs = [0.1, 0.25, 0.5]
num_of_datasets_per_combination = 50
n_iter = 1000
dic = {}
dic['param'] = {
    "num_of_datasets_per_combination":num_of_datasets_per_combination,
    "n_iter": n_iter,
    "n_biomarkers": 10
}

In [40]:
def obtain_ml_order_basic(comb_str, m, dic, n_iter):
    data_dir = f"synthetic/{comb_str}_{m}_reformatted.csv"  
    X, y, bmname, cname = datasets.load_synthetic(data_dir)
    # Fit GMM for each biomarker and plot the results
    mixture_models = mixture_model.fit_all_kde_models(X, y)
    res = mcmc.mcmc(X, mixture_models, n_iter=n_iter, greedy_n_iter=10, greedy_n_init=5)
    # assume our reformated data follows the order of BM1, BM2, BM3, ...
    # their ml_order is in this way
    # for example, [2 1 4 3 0]
    # means that BM3 (i.e., 2+1) is in order 1, 
    # BM2 (1+1) is in order 2, BM5(4+1) is in order 3, BM4(3+1) is in order 4
    # and the real order should be [0, 1, 2, 3, 4] based on our reformated data
    res.sort(reverse=True)
    ml_order = res[0].ordering  
    tau, p_value = kendalltau(ml_order, range(0, len(ml_order)))
    dic[comb_str].append(tau)

In [41]:
for n in ns:
    for r in rs:
        comb_str = f"{int(n*r)}|{n}"
        dic[comb_str] = []
        print(dic)
        for m in range(0, num_of_datasets_per_combination):
            obtain_ml_order_basic(comb_str, m, dic, n_iter=n_iter)

{'param': {'num_of_datasets_per_combination': 50, 'n_iter': 1000, 'n_biomarkers': 10}, '5|50': []}


100%|██████████| 10/10 [00:00<00:00, 5265.92it/s]
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/kde_ebm/mcmc/mcmc.py:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()
100%|██████████| 1000/1000 [00:00<00:00, 8525.67it/s]


{'param': {'num_of_datasets_per_combination': 50, 'n_iter': 1000, 'n_biomarkers': 10}, '5|50': [0.3333333333333333, 0.7333333333333333, 0.3333333333333333, 0.7777777777777777, 0.1111111111111111, 0.06666666666666667, -0.19999999999999998, 0.3333333333333333, 0.3333333333333333, 0.24444444444444444, 0.37777777777777777, 0.6444444444444444, 0.6, 0.4222222222222222, -0.022222222222222223, -0.022222222222222223, 0.37777777777777777, 0.4666666666666666, 0.7777777777777777, 0.37777777777777777, 0.5555555555555555, 0.6888888888888888, 0.6444444444444444, -0.15555555555555553, 0.3333333333333333, 0.4222222222222222, 0.7777777777777777, 0.24444444444444444, 0.5111111111111111, 0.6, 0.19999999999999998, 0.4222222222222222, 0.5555555555555555, 0.19999999999999998, 0.19999999999999998, 0.4666666666666666, 0.19999999999999998, -0.06666666666666667, -0.28888888888888886, 0.4222222222222222, 0.022222222222222223, 0.7333333333333333, 0.19999999999999998, 0.5111111111111111, 0.06666666666666667, -0.111

100%|██████████| 1000/1000 [00:00<00:00, 9637.25it/s]


{'param': {'num_of_datasets_per_combination': 50, 'n_iter': 1000, 'n_biomarkers': 10}, '5|50': [0.3333333333333333, 0.7333333333333333, 0.3333333333333333, 0.7777777777777777, 0.1111111111111111, 0.06666666666666667, -0.19999999999999998, 0.3333333333333333, 0.3333333333333333, 0.24444444444444444, 0.37777777777777777, 0.6444444444444444, 0.6, 0.4222222222222222, -0.022222222222222223, -0.022222222222222223, 0.37777777777777777, 0.4666666666666666, 0.7777777777777777, 0.37777777777777777, 0.5555555555555555, 0.6888888888888888, 0.6444444444444444, -0.15555555555555553, 0.3333333333333333, 0.4222222222222222, 0.7777777777777777, 0.24444444444444444, 0.5111111111111111, 0.6, 0.19999999999999998, 0.4222222222222222, 0.5555555555555555, 0.19999999999999998, 0.19999999999999998, 0.4666666666666666, 0.19999999999999998, -0.06666666666666667, -0.28888888888888886, 0.4222222222222222, 0.022222222222222223, 0.7333333333333333, 0.19999999999999998, 0.5111111111111111, 0.06666666666666667, -0.111

...

100%|██████████| 1000/1000 [00:00<00:00, 7028.93it/s]


{'param': {'num_of_datasets_per_combination': 50, 'n_iter': 1000, 'n_biomarkers': 10}, '5|50': [0.3333333333333333, 0.7333333333333333, 0.3333333333333333, 0.7777777777777777, 0.1111111111111111, 0.06666666666666667, -0.19999999999999998, 0.3333333333333333, 0.3333333333333333, 0.24444444444444444, 0.37777777777777777, 0.6444444444444444, 0.6, 0.4222222222222222, -0.022222222222222223, -0.022222222222222223, 0.37777777777777777, 0.4666666666666666, 0.7777777777777777, 0.37777777777777777, 0.5555555555555555, 0.6888888888888888, 0.6444444444444444, -0.15555555555555553, 0.3333333333333333, 0.4222222222222222, 0.7777777777777777, 0.24444444444444444, 0.5111111111111111, 0.6, 0.19999999999999998, 0.4222222222222222, 0.5555555555555555, 0.19999999999999998, 0.19999999999999998, 0.4666666666666666, 0.19999999999999998, -0.06666666666666667, -0.28888888888888886, 0.4222222222222222, 0.022222222222222223, 0.7333333333333333, 0.19999999999999998, 0.5111111111111111, 0.06666666666666667, -0.111

100%|██████████| 1000/1000 [00:00<00:00, 6903.98it/s]


{'param': {'num_of_datasets_per_combination': 50, 'n_iter': 1000, 'n_biomarkers': 10}, '5|50': [0.3333333333333333, 0.7333333333333333, 0.3333333333333333, 0.7777777777777777, 0.1111111111111111, 0.06666666666666667, -0.19999999999999998, 0.3333333333333333, 0.3333333333333333, 0.24444444444444444, 0.37777777777777777, 0.6444444444444444, 0.6, 0.4222222222222222, -0.022222222222222223, -0.022222222222222223, 0.37777777777777777, 0.4666666666666666, 0.7777777777777777, 0.37777777777777777, 0.5555555555555555, 0.6888888888888888, 0.6444444444444444, -0.15555555555555553, 0.3333333333333333, 0.4222222222222222, 0.7777777777777777, 0.24444444444444444, 0.5111111111111111, 0.6, 0.19999999999999998, 0.4222222222222222, 0.5555555555555555, 0.19999999999999998, 0.19999999999999998, 0.4666666666666666, 0.19999999999999998, -0.06666666666666667, -0.28888888888888886, 0.4222222222222222, 0.022222222222222223, 0.7333333333333333, 0.19999999999999998, 0.5111111111111111, 0.06666666666666667, -0.111

100%|██████████| 1000/1000 [00:00<00:00, 7004.87it/s]


In [42]:
tau_json = 'tau_basic.json'
with open(tau_json, 'w') as fp:
        json.dump(dic, fp, indent = 4)